## Problem Statement

We have been given a PDE: 

$$
\frac{d^2x(t)}{dt^2} = - \dfrac{M}{[(x(t) - x_0)^2 + (y(t) - y_0)^2]^{3/2}}(x(t) - x_0) \\
\frac{d^2y(t)}{dt^2} = - \dfrac{M}{[(x(t) - x_0)^2 + (y(t) - y_0)^2]^{3/2}}(y(t) - y_0)
$$

and boundary condition: 

$$
x_0 = 5\\
y_0 = 6
$$

- Independent variables: t (input)
- Dependent variables: x,y (outputs)

We have to find out $u_x(t)$ and $u_y(t)$ for all t in range [0,20]

Transformando em PINNs:

$$
\frac{d^2N_x(t)}{dt^2} = - \dfrac{M}{[(N_x(t) - x_0)^2 + (N_y(t) - y_0)^2]^{3/2}}(N_x(t) - x_0) \\
\frac{d^2N_y(t)}{dt^2} = - \dfrac{M}{[(N_x(t) - x_0)^2 + (N_y(t) - y_0)^2]^{3/2}}(N_y(t) - y_0)
$$


In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def build_model(input_dimension, hidden_dimension, output_dimension):

    modules=[]
    modules.append(torch.nn.Linear(input_dimension, hidden_dimension[0]))
    modules.append(torch.nn.Tanh())
    for i in range(len(hidden_dimension)-1):
        modules.append(torch.nn.Linear(hidden_dimension[i], hidden_dimension[i+1]))
        modules.append(torch.nn.Tanh())
    
    modules.append(torch.nn.Linear(hidden_dimension[-1], output_dimension))
    
    model = torch.nn.Sequential(*modules)

    return model

In [3]:
net_x = build_model(1, [100, 100, 50, 50], 1)
net_x = net_x.to(device)
mse_cost_function = torch.nn.MSELoss()
optimizer_x = torch.optim.Adam(net_x.parameters())

In [4]:
net_y = build_model(1, [100, 100, 50, 50], 1)
net_y = net_y.to(device)
mse_cost_function = torch.nn.MSELoss()
optimizer_y = torch.optim.Adam(net_y.parameters())

In [5]:
def get_derivativex(x, t, n=1):
    
    if n == 0:
        return x
    else:
        ones = torch.ones(t.size()[0], 1, device=device)
        zeros = torch.zeros(t.size()[0],1, device=device)
        v1 = torch.cat([zeros], dim=1)
        dxdt1 = torch.autograd.grad(x, t, zeros, create_graph=True,
                                retain_graph=True, allow_unused=True)[0]

    return get_derivativex(dxdt1, t, n - 1)

In [6]:
def get_derivativey(y, t, n=1):
    
    if n == 0:
        return y
    else:
        ones = torch.ones(t.size()[0], 1, device=device)
        zeros = torch.zeros(t.size()[0],1, device=device)
        v1 = torch.cat([zeros], dim=1)
        dydt1 = torch.autograd.grad(y, t, zeros, create_graph=True, retain_graph=True, allow_unused=True)[0]

    return get_derivativex(dydt1, t, n - 1)

In [7]:
def f(t, net_x, net_y, y0, x0, M):
    u_x = net_x(t)
    u_y = net_y(t)
    dudtdt_x = get_derivativex(u_x, t, 2)
    denominator = ((u_x - x0)**2 + (u_y - y0)**2)**(1.5)
    pde_x = dudtdt_x + M*(u_x - x0)/denominator

    dudtdt_y = get_derivativey(u_y, t, 2)
    pde_y = dudtdt_y + M*(u_y - y0)/denominator
    return [pde_x, pde_y]


In [8]:
init_x = torch.tensor([[5]], dtype=torch.float32, device=device)
init_y = torch.tensor([[6]], dtype=torch.float32, device=device)
init_v = torch.tensor([[-3]], dtype=torch.float32, device=device)
init_t = torch.tensor([[0]], dtype=torch.float32, device=device, requires_grad=True)

M = torch.tensor(200, dtype=torch.float32, device=device)

n_points = 1000
t_collocation = np.linspace(0, 20, n_points).reshape(-1,1)
all_zeros = np.zeros((n_points,1))

pt_t_collocation = Variable(torch.from_numpy(t_collocation).float(), requires_grad=True).to(device)
pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=True).to(device)

In [9]:
iterations = 50000
loss_count_x = []
loss_boundary = []
loss_func = []
loss_count_y = []

for epoch in range(iterations):
    optimizer_x.zero_grad() 
    init_pred_x = net_x(init_t)
    mse_b_x = (init_pred_x - init_x).norm()
    init_pred_v_x = get_derivativex(net_x(init_t), init_t)
    mse_b_x += (init_pred_v_x - init_v).norm()

    f_out_x = f(pt_t_collocation, net_x, net_y, init_y, init_x, M)[0]
    mse_f_x = mse_cost_function(f_out_x, pt_all_zeros)
    mse_f_x = f_out_x.norm()

    loss_x = mse_b_x + mse_f_x
    loss_count_x.append(loss_x.item())

    loss_x.backward()
    optimizer_x.step()

    
    optimizer_y.zero_grad()
    init_pred_y = net_y(init_t)
    mse_b_y = (init_pred_y - init_y).norm()
    init_pred_v_y = get_derivativey(net_y(init_t), init_t)

    f_out_y = f(pt_t_collocation, net_x, net_y, init_y, init_x, M)[1]
    mse_f_y = mse_cost_function(f_out_y, pt_all_zeros)
    mse_f_y = f_out_y.norm()
    loss_y = mse_b_y + mse_f_y
    loss_count_y.append(loss_y.item())

    loss_y.backward()
    optimizer_y.step()

    with torch.autograd.no_grad():
        if epoch % 1000 == 0:
    	    print(epoch,f"Traning Loss x {loss_x.data}, Training Loss y {loss_y.data}")

0 Traning Loss x 72.99470520019531, Training Loss y 80.28311157226562
1000 Traning Loss x 3.0031354427337646, Training Loss y 14.384612083435059
2000 Traning Loss x 3.0023036003112793, Training Loss y 12.51705551147461
4000 Traning Loss x 3.0007784366607666, Training Loss y 11.577972412109375
5000 Traning Loss x 3.000706195831299, Training Loss y 11.387454986572266
6000 Traning Loss x 3.018238067626953, Training Loss y 11.287607192993164
7000 Traning Loss x 3.0016539096832275, Training Loss y 11.222811698913574
8000 Traning Loss x 3.0072288513183594, Training Loss y 11.180908203125
9000 Traning Loss x 3.0046675205230713, Training Loss y 11.183561325073242
10000 Traning Loss x 3.002288579940796, Training Loss y 11.13392448425293
11000 Traning Loss x 3.011444568634033, Training Loss y 11.119384765625
12000 Traning Loss x 3.0018131732940674, Training Loss y 11.107919692993164
13000 Traning Loss x 3.000669479370117, Training Loss y 11.09931468963623
14000 Traning Loss x 3.0029282569885254,

In [10]:
import plotly.graph_objects as go

xx_test = f_out_x[:, 0].data.cpu().numpy()
yy_test = f_out_y[:, 0].data.cpu().numpy()

fig = go.Figure()
fig.add_scatter(x=xx_test,y=yy_test)

In [11]:
fig = go.Figure()
fig.add_scatter(x=xx_test)

In [12]:
fig = go.Figure()
fig.add_scatter(y=yy_test)

In [13]:
fig = go.Figure()
fig.add_scatter(y = loss_count_x)

In [14]:
fig = go.Figure()
fig.add_scatter(y = loss_count_y)